In [ ]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(5) 
    return soup

In [ ]:
def get_data_inshorts_web(soup):
    
    articles = soup.find_all("div",attrs={"class":"news-card z-depth-1"})
    
    # Declaration
    articles_topic = []
    articles_content = []

    for article in articles:
        # Get article's topic
        topic = article.find("span",attrs={"itemprop":"headline"}).get_text()
        articles_topic.append(topic)
        
        # Get article's content
        content = article.find('div',attrs={'itemprop':'articleBody'}).get_text()
        articles_content.append(content) 
        
    return articles_topic, articles_content

In [ ]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'inshorts Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["sports", "world", "politics", "technology", "startup", "entertainment", "science"]
print("Start scraping from inshorts!")
for category in categories:
    driver = webdriver.Chrome()
    driver.get(f"https://www.inshorts.com/en/read/{category}")
    driver.maximize_window()  
    for i in range (1, 20):
        time.sleep(5)
        button_xpath = "//div[@class='clickable unselectable load-more z-depth-1 hoverable']"  
        driver.find_element(by=By.XPATH, value=button_xpath).click()                    
        time.sleep(5)
    html = driver.page_source
    driver.close()
    soup = BeautifulSoup(html,"html.parser")
    topic, content = get_data_inshorts_web(soup)
    insert_data_to_csv(topic, category, content)
print("Finish scraping from inshorts.com!")